In [ ]:
from src.llm.generator import LLamaCppGeneratorComponent

In [ ]:
prompt = "You are a french news reporter"

In [ ]:
%load_ext dotenv

In [ ]:
%dotenv ./.env_local -o

In [ ]:
API_URL = "http://149.102.141.96"

In [ ]:
llama_cpp_generator = LLamaCppGeneratorComponent(
    api_url=API_URL,
    prompt=prompt
)

In [ ]:
llama_cpp_generator._ping_api()

### Download the File

In [ ]:
filename = "news-clusters-2024-12-03-to-2024-12-03.csv"

In [ ]:
BUCKET_NAME = "congonews-clusters"

In [ ]:
from tempfile import NamedTemporaryFile

In [ ]:
from csv import DictReader as csv_reader

In [ ]:
from itertools import groupby

In [ ]:
from unicodedata import normalize

### Prompt

In [ ]:
from pprint import pprint

In [ ]:
from itertools import groupby

In [ ]:
from src.shared.cloud_storage.cloud_storage_non_numpy import BackBlazeCloudStorageCSV

In [ ]:
cloud_storage = BackBlazeCloudStorageCSV(environment="local")

In [ ]:
from src.llm.generator import SUMMARIZATION_PROMPT_TEMPLATE

In [ ]:
summaries = []
def sort_function(x): return x["labels"]

In [ ]:
data = cloud_storage.read_file_as_list(
    bucket_name=BUCKET_NAME, file_name=filename)

sorted_data = sorted(data, key=sort_function)
grouped_data = groupby(sorted_data, key=sort_function)

In [ ]:
grouped_data[0]

In [ ]:
chat_input = llama_cpp_generator.generate_chat_input(
    template_values={"content": content}, prompt_template=SUMMARIZATION_PROMPT_TEMPLATE)
chat_tokens = llama_cpp_generator.apply_chat_template(
            messages=chat_input, add_generation_prompt=True)

In [ ]:
import json
import requests

In [ ]:
headers = {
        "Content-Type": "application/json",
    }
data = {
        "prompt": chat_tokens,
        "n_predict": 512,
        "temperature": 0.3,
        "top_k": 40,
        "top_p": 0.90,
        "stopped_eos": True,
        "repeat_penalty": 1.05,
        "stop": [
            "assistant",
            "<|im_end|>",
        ],
        "seed": 42,
    }
json_data = json.dumps(data)

In [ ]:
response = requests.post(
    f"{llama_cpp_generator.api_url}/completion",
    headers=headers,
    data=json_data,
    timeout=300,
    auth=llama_cpp_generator.auth_header
)

In [ ]:
llama_cpp_generator._ping_api()

In [ ]:
response.text

In [ ]:
import pprint

In [ ]:
pprint.pprint(response.text)

In [ ]:
def generate_response(prompt: str) -> str:
    """
    This function generates response using the Llamma.cpp api
    Args:
        prompt (str): The prompt to generate response from
    Returns:
        str: The generated response
    """
    
    # Send the POST request
    try:
        
        return response.json()["content"]
    except requests.exceptions.RequestException as e:
        raise e

In [ ]:
chat_tokens

In [ ]:
response = llama_cpp_generator.generate_response(chat_tokens)

In [ ]:
titles

In [ ]:
for label, group in grouped_data:
    news_data = list(group)
    titles = [news["title"] for news in news_data]
    urls = [news["url"] for news in news_data]
    content = "\n".join([news["content"] for news in news_data])
    content = normalize("NFKD", content)
    summary = llama_cpp_generator.run(
        template_values={"content": content}, prompt_template=SUMMARIZATION_PROMPT_TEMPLATE)
    news_data = {
        "titles": titles,
        "urls": urls,
        "summary": summary
    }
    summaries.append(news_data)
    print(f" This is the summary for label {label}")
    pprint(summary)
    print(f"Here are the titles for label {label}")
    print(10 * "*")
    for title in titles:
        print(title)
    print(10 * "*")

I will keep both model in the server and use them. I will be benchmackring them.

Chat template without huggingface.

In [ ]:
import json

In [ ]:
with open("news_summaries.json", "w") as f:
    json.dump(summaries, f, ensure_ascii=False)

In [ ]:
summaries
for summary in summaries:
    pprint(summary["summary"])
    for title_ in summary["titles"]:
        print(title_)
    print(10 * "*")

    print(10 * "*")